# Creating a Dataset using Pheme 

**The dataset can be got form:** https://figshare.com/articles/PHEME_dataset_for_Rumour_Detection_and_Veracity_Classification/6392078

In [0]:
import pandas as pd
import json
import re
from os import walk
from tqdm import tqdm

In [0]:
path= "./pheme-rnr-dataset"

In [0]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [0]:
def listofcontent(path):
    folder = []
    files = []
    for (dirpath, dirnames, filenames) in walk(path):
        folder.extend(dirnames)
        files.extend(filenames)
        break
    return folder,files

In [0]:
rfolder,rfile = listofcontent(path)

In [0]:
data = {"text":[],"in_reply_to_status_id":[],"id":[],"favorite_count":[],"source":[],"in_reply_to_screen_name":[],"id_str":[],"retweet_count":[],"in_reply_to_user_id":[],"favorited":[],"lang":[],"created_at":[],"in_reply_to_status_id_str":[],"place_name":[],"country":[],"place_type":[],"user_id":[],"user_verified":[],"user_followers_count":[],"user_friends_count":[],"user_screen_name":[],"user_favourites_count":[],"user_name":[],"user_created_at":[],"user_is_translator":[],"user_mentions":[],"hashtags":[],"urls":[],"tweet_type":[],"label":[]}

In [15]:
index = 0
for folder in tqdm(rfolder):
    subpath = path + "/" + folder
    subfolder,_ = listofcontent(subpath)
    for basefolder in subfolder:
        rumpath = subpath + "/" + basefolder
        rumfolder,_=listofcontent(rumpath)
        for _id in rumfolder:
            idpath = rumpath + "/" + _id
            idfolder,_ = listofcontent(idpath)
            for action in idfolder:
                tweetpath = idpath + "/" + action
                _,tweetfiles = listofcontent(tweetpath)
                for tweetfile in tweetfiles:
                    f = open(tweetpath + "/" + tweetfile,)
                    tweet = json.load(f)
                    #used to handle specific set of data
                    for header in tweet.keys():
                        if header in data.keys():
                            data[header].append(tweet[header])
                    data["user_id"].append(tweet["user"]["id_str"])
                    data["user_verified"].append(tweet["user"]["verified"])
                    data["user_followers_count"].append(tweet["user"]["followers_count"])
                    data["user_friends_count"].append(tweet["user"]["friends_count"])
                    data["user_screen_name"].append(tweet["user"]["screen_name"])
                    data["user_favourites_count"].append(tweet["user"]["favourites_count"])
                    data["user_created_at"].append(tweet["user"]["created_at"])
                    data["user_is_translator"].append(tweet["user"]["is_translator"])
                    if tweet["place"] == None:
                        data["place_name"].append("")
                        data["country"].append("")
                        data["place_type"].append("")
                    else:    
                        data["place_name"].append(tweet["place"]["full_name"])
                        data["country"].append(tweet["place"]["country"])
                        data["place_type"].append(tweet["place"]["place_type"])
                    if tweet["entities"] == None:
                        data["user_mentions"].append("")
                        data["hashtags"].append("")
                        data["urls"].append(0)
                    else:
                        temp = [i["id_str"]+","+i["screen_name"] for i in tweet["entities"]["user_mentions"]]
                        data["user_mentions"].append(len(join(temp)))
                        temp = [i["text"] for i in tweet["entities"]["hashtags"]]
                        data["hashtags"].append(len(temp))
                        data["urls"].append(len(tweet["entities"]["urls"]))
                    data["tweet_type"].append(action)
                    data["label"].append(basefolder)                    

0it [00:00, ?it/s]


In [0]:
df = pd.DataFrame.from_dict(data)
df.to_csv(r'.\pheme-rumor-dataset.csv', index = True,header = True)